## Update of current stock prices

In [102]:

import yfinance as yf
from datetime import datetime
import pandas
# TODO add specialisation for folder from where to take data

def update_ticker_price():

    tickers = pandas.read_csv('./data/securities.csv')

    for index, row in tickers.iterrows():
        ticker = yf.Ticker(row['ticker']).info
        if ticker['quoteType'] == 'EQUITY':
            market_price = ticker['regularMarketOpen']
            current_time = datetime.now().isoformat()
            tickers.at[index,'price'] = market_price
            tickers.at[index,'price_date'] = current_time
        
    tickers.to_csv('./data/securities.csv',index=False)
    print('Updated ticker prices')
    
update_ticker_price()


Updated ticker prices


In [103]:
import os
from dotenv import load_dotenv

load_dotenv()

endpoint = 'https://api.apilayer.com/exchangerates_data/latest'
access_key = os.getenv('APILAYER_ACCESS_TOKEN')

currencies = {}

def get_rates_for_date(base_currency,target_currency):
    # TODO add date to the function
    if currency in currencies:
        response= currencies[currency]
    else:
        params = {
            'base': DEFAULT_CURRENCY,
            'symbols': currency
        }
        headers= {
            "apikey": access_key
        }
        response = requests.get(endpoint, params=params,headers=headers).json()
        print(response)
        currencies[currency] = response
    return currencies[currency] 

In [104]:
import requests
DEFAULT_CURRENCY='EUR'


import pandas
orders = pandas.read_csv('./data/orders.csv')
for index, row in orders.iterrows():
    currency = orders.at[index,'currency']
    
    if currency != DEFAULT_CURRENCY:
        response = get_rates_for_date(DEFAULT_CURRENCY,currency)
        rate = response['rates'][currency]
        # TODO convert at the date of order
    else:
        rate = 1
    orders.at[index,'price_total_to_currency'] = orders.at[index,'price_total'] / rate + orders.at[index,'tax_for_transaction'] / rate
orders.groupby(['ticker']).sum()
# <!-- calculate totals for each ticker -->
# <!-- update data with current prices -->
# <!-- convert data to default_currency (euro) -->
# <!-- calculate total profit -->
# <!-- save data to the table -->


{'success': True, 'timestamp': 1683005523, 'base': 'EUR', 'date': '2023-05-02', 'rates': {'USD': 1.099022}}
{'success': True, 'timestamp': 1683005523, 'base': 'EUR', 'date': '2023-05-02', 'rates': {'RUB': 88.207458}}


/var/folders/nx/w3ff9pp52293hhppfhvt5vqmqqhdss/T/ipykernel_42211/2026920023.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  orders.groupby(['ticker']).sum()


,amount,price_total,tax_for_transaction,price_total_to_currency
ticker,,,,
ADSK,3,-697.9400,2.80,-543.550076
EPAM,1,-622.0000,1.87,-564.256221
FXIT,3,-35991.9999,107.97,-406.814012
PTC,1,-146.1900,0.44,-132.617909
